# Script to generate urban layouts from the citygenerator package
Generates urban layouts based on input parameters:
1. building density $\lambda_p$
2. frontal aspect ratio $\lambda_f$.

An optional greenery density $\lambda_g$ can be defined.

Randomness is generated by 
3. chossing a fractal type for the street layout: "hierarchical", "cascade" or "random".
4. choosing a degree of randomness for street intersections and street widths.
5. choosing a degree of randomness for block heights.

# Contents
[Setup](#Setup)

[Generate layout](#Generate-layout)

[Video of generation process](#Video-of-generation-process)

[Write data](#Write-data)

[Run checks](#Run-checks-for-generated-layout)

---
## Setup

#### Set up libaries and paths

In [ ]:
%matplotlib inline
import sys, os
import math
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('citygenerator')
import citygenerator as cgen

#### Set up domain and resolution

In [ ]:
imax = 200  # cells in x
jtot = 200  # cells in y
kmax = 200  # cells in z
xsize = 200  # domain size in x (in m)
ysize = 200  # domain size in y (in m)
zsize = 200  # domain size in z (in m)

---
## Generate layout
[Back to contents](#Contents)

### _Set layout parameters here_

In [ ]:
lp = 0.4  # Building density (0 to 1)
lf = 0.2  # Frontal area aspect ratio (0 to 1)
lg = 0.1  # Greenery density (0 to 1)
o = 'r'  # "r for 'random', h for 'hierarchical' and c for 'cascade'
lrandom = 0.6  # degree of randomness for layout (0 to 1)
hrandom = 0.4  # degree of randomness for height (0 to 1)

#### Generate fractal blocks

In [ ]:
savesteps=True  # whether to save step by step creation
blocks, greenery, blockgeneration = cgen.generate_layout(xsize, ysize, zsize, imax, jtot, kmax,
                                                         lp, lg, lf, order=o,
                                                         layoutrandom=lrandom, heightrandom=hrandom, 
                                                         margin=5, minwidth=8, minvolume=10,
                                                         savesteps=savesteps)
blockstats = cgen.utils.calculate_blockstats(blocks, a0=xsize*ysize)
print("lamda_p = ", blockstats['planindex'])
print("lambda_f = ", blockstats['frontindex'])

#### Display block layout

In [ ]:
limits=[0, xsize, 0, ysize, 0, zsize/2]
fig = plt.figure(figsize=(8, 4.5), dpi=150)
# geometry plot
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
cgen.plot_3dlayout(blocks, ax=ax1, limits=limits, edgecolor='k')
cgen.plot_3dlayout(greenery, ax=ax1, limits=limits, facecolor='g')
ax1.dist=11
# layout plot
ax2 = fig.add_subplot(1, 2, 2)
cgen.plot_2dlayout(blocks, ax=ax2, limits=limits[:4])
cgen.plot_2dlayout(greenery, ax=ax2, limits=limits[:4], facecolor='g')
plt.tight_layout()
plt.show()

#### Block statistics

In [ ]:
blockstats = cgen.utils.calculate_blockstats(blocks, a0=xsize*ysize)
print("lamda_p = ", blockstats['planindex'])
print("lambda_f = ", blockstats['frontindex'])
print("zmax = ", blockstats['heightmax'])
print("zh = ", blockstats['heightmean'])
print("no. of blocks: ", blockstats['nblocks'])
print("blocks:\n", blocks)

---
## Video of generation process
[Back to contents](#Contents)

In [ ]:
save=False
path="./videos/"
show=True
limits=[0, xsize, 0, ysize, 0, zsize/3]
fig = plt.figure(figsize=(8, 4.5), dpi=150)
cgen.video_layout(blockgeneration, greenery, fig=fig, limits=limits, edgecolor='k', 
                  save=save, path=path, show=show)

---
## Write data
[Back to contents](#Contents)

#### Change resolution or convert blocks to grid indices

In [ ]:
dx = imax/xsize
dy = jtot/ysize
dz = kmax/zsize
block_indices = cgen.utils.convert(blocks, dx, dy, dz, rounding=True)
print(block_indices)

#### Write the data

In [ ]:
savedir = "./layouts"
if not os.path.exists(savedir):
    os.mkdir(savedir)
blocksfile = os.path.join(savedir, 'blocks.txt')
cgen.utils.write(block_indices, blocksfile)

---
## Run checks for generated layout
[Back to contents](#Contents)

In [ ]:
domain=[0, xsize, 0, ysize, 0, zsize]
dx = xsize/imax
dy = ysize/jtot
dz = zsize/kmax
resolution = [dx, dy, dz]
problemblocks = cgen.run_checks(blocks, domain, resolution, heightratio=6, blockvolume=10)
if problemblocks:  # if list is not empty
    fig = plt.figure(figsize=(8, 4.5), dpi=150)
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    cgen.plot_3dlayout(problemblocks, ax=ax1, edgecolor='k', 
                       limits=domain)
    ax2 = fig.add_subplot(1, 2, 2)
    cgen.plot_2dlayout(problemblocks, ax=ax2, limits=domain[:4])
    plt.suptitle("Problematic blocks")    
    plt.tight_layout()
    plt.show()

[Back to contents](#Contents)